In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train_V2.csv', 'test_V2.csv', 'sample_submission_V2.csv']


In [19]:
df_train = pd.read_csv('../input/train_V2.csv', low_memory=False)

# No labels
df_test = pd.read_csv('../input/test_V2.csv', low_memory=False)

In [20]:
dummies_train = pd.get_dummies(df_train['matchType'])
dummies_test = pd.get_dummies(df_test['matchType'])

In [21]:
df_train = pd.concat([df_train, dummies_train], axis=1)
df_test = pd.concat([df_test, dummies_test], axis=1)

In [9]:
df_train

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc,crashfpp,crashtpp,duo,duo-fpp,flarefpp,flaretpp,normal-duo,normal-duo-fpp,normal-solo,normal-solo-fpp,normal-squad,normal-squad-fpp,solo,solo-fpp,squad,squad-fpp
0,0,0,0.000,0,0,0,60,1241,0,0,0.000,1306,28,26,-1,0,0.0000,0,0.000,0,0,244.80,1,1466,0.4444,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,91.470,0,0,0,57,0,0,0,0.000,1777,26,25,1484,0,0.0045,0,11.040,0,0,1434.00,5,0,0.6400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,0,68.000,0,0,0,47,0,0,0,0.000,1318,50,47,1491,0,0.0000,0,0.000,0,0,161.80,2,0,0.7755,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,32.900,0,0,0,75,0,0,0,0.000,1436,31,30,1408,0,0.0000,0,0.000,0,0,202.70,3,0,0.1667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,100.000,0,0,0,45,0,1,1,58.530,1424,97,95,1560,0,0.0000,0,0.000,0,0,49.75,2,0,0.1875,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5,0,0,100.000,1,1,0,44,0,1,1,18.440,1395,28,28,1418,0,0.0000,0,0.000,0,0,34.70,1,0,0.0370,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,0,0,0.000,0,0,0,96,1262,0,0,0.000,1316,28,28,-1,0,0.0000,0,0.000,0,0,13.50,1,1497,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7,0,0,8.538,0,0,0,48,1000,0,0,0.000,1967,96,92,-1,0,2004.0000,0,0.000,0,0,1089.00,6,1500,0.7368,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8,0,0,51.600,0,0,0,64,0,0,0,0.000,1375,28,27,1493,0,0.0000,0,0.000,0,0,799.90,4,0,0.3704,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9,0,0,37.270,0,0,0,74,0,0,0,0.000,1930,29,27,1349,0,0.0000,0,0.000,0,0,65.67,1,0,0.2143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [22]:
df_train.drop(['Id', 'groupId', 'matchId', 'matchType'], axis=1, inplace=True)

In [10]:
df_train.isna().sum()

assists             0
boosts              0
damageDealt         0
DBNOs               0
headshotKills       0
heals               0
killPlace           0
killPoints          0
kills               0
killStreaks         0
longestKill         0
matchDuration       0
maxPlace            0
numGroups           0
rankPoints          0
revives             0
rideDistance        0
roadKills           0
swimDistance        0
teamKills           0
vehicleDestroys     0
walkDistance        0
weaponsAcquired     0
winPoints           0
winPlacePerc        1
crashfpp            0
crashtpp            0
duo                 0
duo-fpp             0
flarefpp            0
flaretpp            0
normal-duo          0
normal-duo-fpp      0
normal-solo         0
normal-solo-fpp     0
normal-squad        0
normal-squad-fpp    0
solo                0
solo-fpp            0
squad               0
squad-fpp           0
dtype: int64

In [23]:
df_train.dropna(0, inplace=True)

In [24]:
feature_cols = df_train.columns[df_train.columns!='winPlacePerc']

X = df_train[feature_cols]
y = df_train.winPlacePerc

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [26]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [27]:
def submission(df, features, algorithm):
    x_oos = df[features]
    algo = algorithm
    algo.fit(X_train, y_train)
    pred = algo.predict(x_oos)
    
    test_id = df["Id"]
    sub = pd.DataFrame({'Id': test_id, "winPlacePerc": pred} , columns=['Id', 'winPlacePerc'])
    return sub.head()

In [28]:
print(submission(df_test, feature_cols, LinearRegression()))

               Id  winPlacePerc
0  9329eb41e215eb   1977.095264
1  639bd0dcd7bda8   9562.140677
2  63d5c8ef8dfe91   2628.691311
3  cf5b81422591d1   5473.064518
4  ee6a295187ba21   9037.646365


In [29]:
'''
Create a submission file
'''
import random

# predict class probabilities for the actual testing data (not X_test)
X_oos = df_test[feature_cols]

svr = LinearRegression()
svr.fit(X_train,y_train)
pred = svr.predict(X_oos)


test_id = df_test["Id"]
sub = pd.DataFrame({'Id': test_id, "winPlacePerc": pred} , columns=['Id', 'winPlacePerc'])
sub.to_csv("submission_V2.csv", index = False)